
Model:
Embedding: **Senstence BERT** (Ref:https://www.sbert.net/)

Processing Model: **Multilayer Perceptron(MLP)** from scikit-learn

Accuracy: 29

Run code thru colab:https://colab.research.google.com/drive/1LZcNvFpkZTdeAbpy8FHvGUHG1GKV0Xna#scrollTo=RAyehqJ5bDpd

In [2]:
import pandas as pd
import numpy as np
pd.set_option('max_rows', 100)
pd.set_option('max_colwidth', 200)
pd.describe_option('max_colwidth')

display.max_colwidth : int or None
    The maximum width in characters of a column in the repr of
    a pandas data structure. When the column overflows, a "..."
    placeholder is embedded in the output. A 'None' value means unlimited.
    [default: 50] [currently: 200]


In [3]:
# Preprocessing
dataset = pd.read_csv('https://raw.githubusercontent.com/leadlost/classify-mental-problems-based-on-queries/main/new_dataset.csv')
dataset = pd.DataFrame(dataset, columns = ['idx', 'labels','content'])
dataset.pop('idx')

# Seperate labels
res_list = [] #Array of labels with type int
labels = []
count = 0
class_names = []
for i in range(11):
  name = []
  class_names.append(name)

for row in dataset['labels'].values.tolist():
  label = row.replace(',','')
  label = label.replace('[','')
  label = label.replace(']','')
  label = label.split() #Split to List
  row_res = []
  for ele in label:
    ele = int(ele)
    row_res.append(ele) 
  row_res = pd.Series(row_res)
  res_list.append(row_res)
 
  for i in range(len(label)):
    val = label[i]
    val = int(val)
    class_names[i].append(val)

for i in range(len(class_names)):
  name = 'class'+str(i)
  dataset[name] = class_names[i]


In [4]:
newlist = dataset['labels'].values.tolist()
print(len(set(newlist)))


87


In [5]:
print(dataset.columns)
print(type(res_list))
dataset['res_list'] = res_list
print(dataset.columns)

Index(['labels', 'content', 'class0', 'class1', 'class2', 'class3', 'class4',
       'class5', 'class6', 'class7', 'class8', 'class9', 'class10'],
      dtype='object')
<class 'list'>
Index(['labels', 'content', 'class0', 'class1', 'class2', 'class3', 'class4',
       'class5', 'class6', 'class7', 'class8', 'class9', 'class10',
       'res_list'],
      dtype='object')


In [6]:
# SBERT Embedding
%%time
!pip install -U sentence-transformers
from numpy.ma.core import shape
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = []
for row in dataset['content'].values.tolist():
  embedding = model.encode(row)
  embeddings.append(embedding)
dataset['embeddings'] = embeddings
print(dataset.columns)

     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 4.0 MB 36.6 MB/s 
     |████████████████████████████████| 1.2 MB 12.1 MB/s 
     |████████████████████████████████| 77 kB 3.5 MB/s 
     |████████████████████████████████| 6.6 MB 28.6 MB/s 
     |████████████████████████████████| 596 kB 50.1 MB/s 
     |████████████████████████████████| 895 kB 65.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=365ebffe17d350b823e3110bebed65c62a8fe5ae149b31c3040bd9a187edeead
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Index(['labels', 'content', 'class0', 'class1', 'class2', 'class3', 'class4',
       'class5', 'class6', 'class7', 'class8', 'class9', 'class10', 'res_list',
       'embeddings'],
      dtype='object')
CPU times: user 51.1 s, sys: 2.53 s, total: 53.7 s
Wall time: 1min 20s


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
X = dataset['embeddings'].values.tolist()
Y = dataset['class1']
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2, random_state = 1)#random_state = 1
features = []
print(testY.index.values.tolist())
for idx in testY.index.values.tolist():
  features.append(res_list[idx])
print(len(features))

[625, 318, 339, 598, 90, 666, 262, 84, 585, 85, 567, 664, 216, 189, 19, 273, 314, 375, 60, 550, 600, 59, 488, 432, 632, 414, 47, 388, 544, 738, 559, 181, 310, 457, 667, 285, 506, 740, 187, 104, 101, 663, 8, 331, 117, 69, 255, 493, 522, 486, 402, 529, 201, 265, 716, 81, 394, 378, 411, 434, 148, 56, 289, 502, 736, 479, 23, 195, 596, 683, 671, 656, 154, 651, 579, 323, 202, 725, 591, 50, 528, 626, 120, 495, 224, 428, 669, 175, 185, 159, 435, 61, 214, 540, 382, 597, 3, 622, 441, 134, 329, 65, 593, 180, 111, 602, 57, 543, 419, 355, 577, 358, 403, 268, 247, 35, 383, 710, 334, 527, 41, 551, 630, 16, 556, 108, 617, 747, 379, 0, 241, 74, 293, 66, 385, 301, 496, 696, 363, 691, 555, 257, 745, 421, 13, 452, 739, 519, 92, 472]
150


In [8]:
# MLP CLASSIFIER
max_iter = 100
mlp = MLPClassifier(    
                    max_iter = max_iter,
                    solver =  'adam',
                    activation = 'relu', 
                    early_stopping= True, 
                    random_state = 5,
                    hidden_layer_sizes = (150,)
                    )
# TRAIN
mlp.fit(trainX, trainY)
print(mlp.get_params())
# Pred
trainPred = mlp.predict(trainX)
testPred = mlp.predict(testX)
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report, confusion_matrix

# plt.plot(mlp.loss_curve_)
# print('Training set accuracy', mlp.score(trainX, trainY))
# print('Development set accuracy', mlp.score(testX, testY))
# print(confusion_matrix(testY, testPred))
# print(classification_report(testY, testPred)) # output_dict=True, makes the output a dict
# print(accuracy_score(testY, testPred))
# print(accuracy_score(testY, testPred, normalize = False))# num of the correct prediction
print(testY)

{'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': True, 'epsilon': 1e-08, 'hidden_layer_sizes': (150,), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_fun': 15000, 'max_iter': 100, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': 5, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
625    0
318    0
339    0
598    0
90     0
      ..
452    0
739    0
519    0
92     1
472    0
Name: class1, Length: 150, dtype: int64


In [9]:
true_results = []
accuracies = []
testPreds = []
for i in range(11):
  X = dataset['embeddings'].values.tolist()
  name = 'class'+str(i)
  Y = dataset[name]
  # Split train test
  trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2, random_state = 1)
  true_results.append(testY) 
  # TRAIN
  mlp.fit(trainX, trainY)
  # Predict
  testPred = mlp.predict(testX)
  testPreds.append(testPred.tolist())
  # Evaluate
  accuracy = accuracy_score(testY, testPred)
  accuracies.append(accuracy)
print(accuracies)
print(len(true_results))

[0.8666666666666667, 0.8533333333333334, 0.9066666666666666, 0.8533333333333334, 0.8, 0.9066666666666666, 0.9333333333333333, 0.96, 0.9466666666666667, 0.9, 0.9466666666666667]
11


In [11]:
from pandas.core.frame import DataFrame
# Label: true_results
# Predictions: df_testPreds

df_testPreds = pd.DataFrame(testPreds)
true_results = pd.DataFrame(true_results)

print(df_testPreds.shape)
print(true_results.shape)
df_testPreds_list = []
true_results_list = []
lenrange = len(df_testPreds.columns)

for i in range(lenrange):
  df_testPreds_list.append(str(df_testPreds.iloc[:,i].values.tolist()))
  true_results_list.append(str(true_results.iloc[:,i].values.tolist()))

print('-------------------------------------------------------------------------')
print(df_testPreds_list[0])
print('-------------------------------------------------------------------------')
print(true_results_list[0])

# df_testPreds_list = df_testPreds.values.flatten()
# true_results_list = true_results.values.flatten()
# print(classification_report(true_results_list, df_testPreds_list))
from sklearn.metrics import f1_score
print('macro avg: ',f1_score(true_results_list, df_testPreds_list, average='macro'))
print('micro avg: ',f1_score(true_results_list, df_testPreds_list, average='micro'))
print('weighted avg: ',f1_score(true_results_list, df_testPreds_list, average='weighted'))
print(len(set(df_testPreds_list)))
print(len(set(true_results_list)))
# df_testPreds = df_testPreds.values.flatten()
# true_results = true_results.values.flatten()
# print(classification_report(true_results, df_testPreds))
# from sklearn.metrics import f1_score
# print('macro avg: ',f1_score(true_results, df_testPreds, average='macro'))
# print('micro avg: ',f1_score(true_results, df_testPreds, average='micro'))
# print('weighted avg: ',f1_score(true_results, df_testPreds, average='weighted'))

(11, 150)
(11, 150)
-------------------------------------------------------------------------
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
-------------------------------------------------------------------------
[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
macro avg:  0.0645247796911634
micro avg:  0.29333333333333333
weighted avg:  0.23402991349341093
14
47
